# Capstone: Financial Research Copilot

## Step 0: Setup and Configuration
Importing necessary libraries (yfinance, pandas, google-generativeai) and configuring the Google Gemini client using the API key from `.env`.

In [58]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import google.generativeai as genai
import json
from ddgs import DDGS

In [2]:
# 1. Load environment variables
load_dotenv()

# 2. Get API Key
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    print("Error: GOOGLE_API_KEY not found in .env file.")
else:
    print("API Key loaded successfully.")
    
    # 3. Configure the client
    try:
        genai.configure(api_key=api_key)
        print("Google GenAI Client configured successfully.")
    except Exception as e:
        print(f"Error configuring client: {e}")

API Key loaded successfully.
Google GenAI Client configured successfully.


## Step 1.1: Price History Tool
Defining `get_asset_price_history` to fetch OHLCV data for Stocks and ETFs using `yfinance`. This tool returns structured JSON data that the agent can analyze.


In [3]:
from typing import Dict, Any

def get_asset_price_history(symbol: str, asset_type: str = "equity", period: str = "1mo") -> Dict[str, Any]:
    """
    Fetch historical OHLCV price data for an asset.
    
    Args:
        symbol (str): Ticker symbol (e.g., 'AAPL', 'BTC-USD').
        asset_type (str): Type of asset ('equity', 'crypto', 'etf'). Defaults to 'equity'.
        period (str): Data period to download (e.g., '1mo', '3mo', '1y'). Defaults to '1mo'.
        
    Returns:
        dict: Structured data containing status and price history.
    """
    try:
        # Handle crypto symbols for yfinance (usually needs -USD suffix if not provided)
        if asset_type.lower() == 'crypto' and not symbol.endswith('-USD'):
            symbol = f"{symbol}-USD"
            
        ticker = yf.Ticker(symbol)
        
        # Fetch history
        hist = ticker.history(period=period)
        
        if hist.empty:
            return {
                "status": "error",
                "message": f"No data found for {symbol}. Check ticker or period."
            }
            
        # Format data for the agent (convert to list of dicts)
        # We limit to the last 60 records to keep context small if period is long
        data_records = []
        for date, row in hist.tail(60).iterrows():
            data_records.append({
                "date": date.strftime('%Y-%m-%d'),
                "close": round(row['Close'], 2),
                "volume": int(row['Volume'])
            })
            
        return {
            "status": "success",
            "symbol": symbol,
            "currency": ticker.info.get('currency', 'USD'),
            "data": data_records
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}

In [ ]:
# --- Testing the tool  ---
print("Testing Tool 1.1 with AAPL...")
result = get_asset_price_history("AAPL", period="5d")
result = json.dumps(result, indent=4)
print(result)


Testing Tool 1.1 with AAPL...
{'status': 'success', 'symbol': 'AAPL', 'currency': 'USD', 'data': [{'date': '2025-11-20', 'close': np.float64(266.25), 'volume': 45823600}, {'date': '2025-11-21', 'close': np.float64(271.49), 'volume': 59030800}, {'date': '2025-11-24', 'close': np.float64(275.92), 'volume': 65585800}, {'date': '2025-11-25', 'close': np.float64(276.97), 'volume': 46914200}, {'date': '2025-11-26', 'close': np.float64(277.55), 'volume': 33431400}]}


## Step 1.2: Fundamentals Tool
Defining `get_asset_fundamentals` to retrieve key financial metrics (P/E, Market Cap, Sector) using `yfinance`. This helps the agent assess value and risk.


In [5]:
def get_asset_fundamentals(symbol: str, asset_type: str = "equity") -> Dict[str, Any]:
    """
    Fetch fundamental data (P/E, Market Cap, Sector, Summary) for an asset.
    
    Args:
        symbol (str): Ticker symbol (e.g., 'AAPL').
        asset_type (str): 'equity' or 'etf'. (Crypto has limited fundamentals in yf).
        
    Returns:
        dict: Key fundamentals or error message.
    """
    try:
        if asset_type.lower() == 'crypto' and not symbol.endswith('-USD'):
             symbol = f"{symbol}-USD"

        ticker = yf.Ticker(symbol)
        info = ticker.info
        
        # Extract only the most relevant fields to save context tokens
        fundamentals = {
            "symbol": symbol,
            "name": info.get("shortName", "Unknown"),
            "sector": info.get("sector", "Unknown"),
            "industry": info.get("industry", "Unknown"),
            "market_cap": info.get("marketCap", "N/A"),
            "trailing_pe": info.get("trailingPE", "N/A"),
            "forward_pe": info.get("forwardPE", "N/A"),
            "dividend_yield": info.get("dividendYield", "N/A"),
            "fifty_two_week_high": info.get("fiftyTwoWeekHigh", "N/A"),
            "fifty_two_week_low": info.get("fiftyTwoWeekLow", "N/A"),
            "summary": info.get("longBusinessSummary", "No summary available")[:300] + "..." # Truncate summary
        }
        
        return {
            "status": "success",
            "data": fundamentals
        }
        
    except Exception as e:
        return {"status": "error", "message": str(e)}


In [ ]:
# --- Test the tool immediately ---
print("Testing Tool 1.2 with MSFT...")
fund_result = get_asset_fundamentals("MSFT")
fund_result = json.dumps(fund_result, indent=4)
print(fund_result)

Testing Tool 1.2 with MSFT...
{'status': 'success', 'data': {'symbol': 'MSFT', 'name': 'Microsoft Corporation', 'sector': 'Technology', 'industry': 'Software - Infrastructure', 'market_cap': 3608802230272, 'trailing_pe': 34.481533, 'forward_pe': 32.47492, 'dividend_yield': 0.75, 'fifty_two_week_high': 555.45, 'fifty_two_week_low': 344.79, 'summary': "Microsoft Corporation develops and supports software, services, devices, and solutions worldwide. The company's Productivity and Business Processes segment offers Microsoft 365 Commercial, Enterprise Mobility + Security, Windows Commercial, Power BI, Exchange, SharePoint, Microsoft Teams, Security a..."}}


## Step 1.3: Technical Analysis Tool
Defining `calculate_technical_indicators` to compute Volatility (Standard Deviation) and Moving Averages. This gives the agent quantitative data to assess risk.


In [7]:
def calculate_technical_indicators(symbol: str, period: str = "3mo") -> Dict[str, Any]:
    """
    Calculate technical indicators (Volatility, SMA) for a symbol.
    
    Args:
        symbol (str): Ticker symbol.
        period (str): Data lookback period (default '3mo').
        
    Returns:
        dict: Calculated metrics including annualized volatility and current trend.
    """
    try:
        # We reuse our first tool logic to get the raw dataframe usually, 
        # but calling yfinance directly here is cleaner for the standalone tool.
        ticker = yf.Ticker(symbol)
        hist = ticker.history(period=period)
        
        if hist.empty:
             return {"status": "error", "message": "No data for calculations."}
        
        # 1. Calculate Daily Returns
        hist['Returns'] = hist['Close'].pct_change()
        
        # 2. Annualized Volatility (Standard Deviation of returns * sqrt(252 trading days))
        volatility = hist['Returns'].std() * np.sqrt(252)
        
        # 3. Simple Moving Average (50-day) - requiring enough data
        if len(hist) >= 50:
            sma_50 = hist['Close'].rolling(window=50).mean().iloc[-1]
        else:
            sma_50 = "Not enough data"
            
        current_price = hist['Close'].iloc[-1]
        
        # Determine trend
        trend = "Neutral"
        if isinstance(sma_50, (int, float)):
            if current_price > sma_50:
                trend = "Uptrend (Above 50d SMA)"
            else:
                trend = "Downtrend (Below 50d SMA)"
        
        return {
            "status": "success",
            "symbol": symbol,
            "current_price": round(current_price, 2),
            "annualized_volatility": round(volatility * 100, 2), # as percentage
            "sma_50": round(sma_50, 2) if isinstance(sma_50, float) else sma_50,
            "trend": trend
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}


In [ ]:
# --- Test the tool ---
print("Testing Tool 1.3 with NVDA...")
tech_result = calculate_technical_indicators("NVDA")
tech_result = json.dumps(tech_result, indent=4)
print(tech_result)

Testing Tool 1.3 with NVDA...
{'status': 'success', 'symbol': 'NVDA', 'current_price': np.float64(180.26), 'annualized_volatility': np.float64(37.57), 'sma_50': np.float64(186.81), 'trend': 'Downtrend (Below 50d SMA)'}


## Step 1.4: Market Opportunity Scanner Tool
Defining `scan_market_opportunities` to filter a watchlist of major assets for potential "Buys". It looks for:
1. **Value**: Low P/E ratio (< 25).
2. **Momentum**: Strong recent performance.
3. **Dip Buys**: Assets down significantly from their 52-week high.


In [59]:
def search_market_trends(query: str) -> Dict[str, Any]:
    """
    Search the web for financial news, trending stocks, or market analysis.
    Use this to find NEW ticker ideas (e.g., "undervalued AI stocks 2025").
    
    Args:
        query (str): Search query (e.g., "best biotech stocks under $50").
        
    Returns:
        dict: Top 5 search results with titles, snippets, and URLs.
    """
    try:
        results = []
        # specific 'news' backend is often good for markets, but 'text' is broader
        with DDGS() as ddgs:
            # Fetch top 5 results
            ddg_results = list(ddgs.text(query, max_results=5))
            
            if not ddg_results:
                return {"status": "error", "message": "No results found."}
            
            for r in ddg_results:
                results.append({
                    "title": r.get('title'),
                    "snippet": r.get('body'), # DDGS uses 'body' for snippet
                    "link": r.get('href')
                })
                
        return {
            "status": "success",
            "query": query,
            "results": results
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}

In [60]:
# --- Test ---
print("Searching for Trending Stocks...")
print(search_market_trends("fastest growing AI stocks 2025"))

Searching for Trending Stocks...
{'status': 'success', 'query': 'fastest growing AI stocks 2025', 'results': [{'title': '6 Best Artificial Intelligence (AI) Stocks To Buy In 2025 ...', 'snippet': 'Dec 16, 2024 · Below is a brief overview of the state of AI , followed by a list of six AI stocks that look promising for 2025 .', 'link': 'https://www.forbes.com/sites/investor-hub/article/best-artificial-intelligence-ai-stocks/'}, {'title': '4 AI Stocks Poised for Explosive Growth in 2025 - Are You In?', 'snippet': 'Mar 6, 2025 · In this evolving AI landscape, several companies are positioned to lead the revolution. These include NVIDIA NVDA, Fortinet FTNT, C3.ai AI and Palantir Technologies PLTR, each playing a unique...', 'link': 'https://www.nasdaq.com/articles/4-ai-stocks-poised-explosive-growth-2025-are-you'}, {'title': 'Top 5 AI Stocks That Will Skyrocket in 2025 - Analytics Insight', 'snippet': 'Jul 7, 2025 · The five top AI stocks expected to perform very well in 2025 are Nvidia, AM

## Step 2: Initialize Agent with Tools
We define the `tools` list containing our three functions and configure the Gemini Pro model to use them. We then start a chat session (Agent) that knows about these tools.


In [16]:
# 1. Define the Tool List
# We pass the actual function objects to the GenAI SDK. 
# The SDK automatically inspects the docstrings to understand how to use them.
my_tools = [
    get_asset_price_history,
    get_asset_fundamentals,
    calculate_technical_indicators
]

# 2. Initialize the Model with Tools
# We use 'gemini-1.5-flash' or 'gemini-1.5-pro' for best tool performance.
# 'auto' function calling mode is default, meaning the model decides when to use tools.
model = genai.GenerativeModel(
    model_name='gemini-2.5-flash', # Or 'gemini-1.5-pro' if you have access
    tools=my_tools
)

# 3. Create the Agent (Chat Session)
# enable_automatic_function_calling=True means the SDK handles the loop:
# Model asks for tool -> SDK runs tool -> SDK gives answer back to Model -> Model answers user.
agent = model.start_chat(enable_automatic_function_calling=True)



In [19]:
print("Agent initialized with Financial Tools!")

# --- Quick Test ---
# We ask a question that REQUIRES data to answer.
print("\nAsking Agent: 'What is the current volatility of Apple and is it in an uptrend?'")
response = agent.send_message("What is the current volatility of Apple (AAPL) and is it in an uptrend?")

print("\nAgent Answer:")
print(response.text)

Agent initialized with Financial Tools!

Asking Agent: 'What is the current volatility of Apple and is it in an uptrend?'

Agent Answer:
The current annualized volatility of Apple (AAPL) is 22.72%. Yes, it is in an uptrend as its current price of $277.55 is above its 50-day Simple Moving Average (SMA) of $261.63.


## Step 3: Defining the Financial Advisor Persona
We re-initialize the agent with a **System Instruction**. This tells the model to act as a professional Investment Advisor, mandates that it *must* use tools before answering, and defines risk rules (e.g., "High volatility > 30% requires a warning").


In [29]:
# Define the System Prompt (Persona & Rules)
advisor_instruction = advisor_instruction = """
You are an expert Financial Research Assistant and Investment Advisor.

YOUR GOAL:
Help the user analyze assets and construct portfolios based on data.

RULES:
1. ALWAYS fetch real data using your tools before answering. Do not guess.
2. If a user asks about a stock, get its Fundamentals AND Technicals.
3. RISK ASSESSMENT:
   - If Annualized Volatility is > 30%, label the asset as "High Risk".
   - If Volatility is < 15%, label it as "Stable/Low Risk".
4. RECOMMENDATIONS: Cite specific metrics to support your view.

FORMATTING REQUIREMENTS:
- You MUST present data in two distinct tables:
  
  **Table 1: Fundamentals**
  | Metric | Value |


  **Table 2: Technicals**
  | Metric | Value |

- Make sure the table is well format in a uniform spaced manner with its borders
- Do not leave tables empty. If data is missing, write "N/A".
"""

# Re-initialize model with the system instruction
advisor_model = genai.GenerativeModel(
    model_name='gemini-2.5-flash',
    tools=my_tools,
    system_instruction=advisor_instruction
)

# Start the Advisor Chat
advisor_agent = advisor_model.start_chat(enable_automatic_function_calling=True)

print("Advisor Agent initialized with System Instructions!")


Advisor Agent initialized with System Instructions!


In [30]:
# --- Test the Persona ---
query = "Analyze Tesla (TSLA). Is it safe for a conservative retiree?"
print(f"\nUser: {query}")
response = advisor_agent.send_message(query)

print("\nAdvisor Response:")
print(response.text)


User: Analyze Tesla (TSLA). Is it safe for a conservative retiree?

Advisor Response:
Here's an analysis of Tesla (TSLA):

**Table 1: Fundamentals**
| Metric            | Value               |
|:------------------|:--------------------|
| Company Name      | Tesla, Inc.         |
| Sector            | Consumer Cyclical   |
| Industry          | Auto Manufacturers  |
| Market Cap        | $1.42 T             |
| Trailing P/E      | 294.19              |
| Forward P/E       | 131.66              |
| Dividend Yield    | N/A                 |
| 52 Week High      | $488.54             |
| 52 Week Low       | $214.25             |

**Table 2: Technicals**
| Metric               | Value            |
|:---------------------|:-----------------|
| Current Price        | $426.58          |
| Annualized Volatility| 50.72%           |
| 50-day SMA           | $433.68          |
| Trend                | Downtrend (Below 50d SMA)|

**Risk Assessment:**
Based on an Annualized Volatility of 50.72%, Te

## Step 4: Portfolio Allocation Tool
Defining `allocate_portfolio` to simulate investment execution. This tool takes a list of assets and a budget, fetches real-time prices, and calculates an equal-weighted allocation (share counts and values).


In [ ]:
# --- FINAL AGENT SETUP (Production Ready) ---

# 1. Consolidate all tools
final_tools = [
    get_asset_price_history,        # Tool 1.1
    get_asset_fundamentals,         # Tool 1.2
    calculate_technical_indicators, # Tool 1.3
    allocate_portfolio,             # Tool 1.4 (Portfolio)
    search_market_trends            # Tool 1.5 (Web Search)
]

# 2. robust System Instruction
final_instruction = """
You are a Strategic Investment Advisor (Powered by Gemini 2.5).

YOUR GOAL:
Build a solid core portfolio AND identify high-potential "Sleeper" stocks.

WORKFLOW:
1. **Discovery**: Search for trending/underrated ideas (`search_market_trends`).
2. **Verification**: Check data for ALL candidates (`get_asset_fundamentals`, `calculate_technical_indicators`).
3. **Selection**:
   - Select the BEST 3-5 assets for the **Main Portfolio** (Balance of Risk/Reward).
   - Select 1-2 assets that are interesting but didn't make the main cut (e.g., higher risk, smaller cap, turnaround play) for the **"Underrated"** list.

OUTPUT SECTIONS (Strictly follow this order):
1. **Market Insight**: "Based on search trends, I focused on [Sector]..."
2. **Core Portfolio Analysis**:
   - Data Tables (Fundamentals & Technicals) for the main picks.
3. **Proposed Allocation**: The `allocate_portfolio` table.
4. **💎 Underrated Stocks to Consider**:
   - List 1-2 stocks that are NOT in the main portfolio above.
   - **Why?**: Explain the "Growth Story" vs "Why it was excluded" (e.g., "High potential due to AI news, but excluded from core due to 85% volatility. Good for a small speculative bet.").
5. **Glossary**: Define technical terms.

DISCLAIMER: "This is a simulation for educational purposes."
"""

# Initialize Model with the specific version
# Ensure your SDK version supports this model name string.
final_model = genai.GenerativeModel(
    model_name='gemini-2.5-flash',  # <--- UPDATED HERE
    tools=final_tools,
    system_instruction=final_instruction
)



In [66]:
# 4. Start Session
analyst_desk = final_model.start_chat(enable_automatic_function_calling=True)

print("Final 'Trend Hunter' Analyst Desk is Ready!")
print("Type 'exit' to stop.\n")

# --- INTERACTIVE LOOP ---
while True:
    user_input = input("👤 You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    
    print("Agent is thinking...")
    try:
        response = analyst_desk.send_message(user_input)
        print(f"\nAnalyst:\n{response.text}\n")
        print("-" * 80)
    except Exception as e:
        print(f"Error: {e}")

Final 'Trend Hunter' Analyst Desk is Ready!
Type 'exit' to stop.

Agent is thinking...


C:\Program Files\Python313\Lib\http\cookiejar.py:810: ResourceWarning: unclosed <ssl.SSLSocket fd=2828, family=2, type=1, proto=0, laddr=('192.168.1.110', 63439), raddr=('52.149.246.39', 443)>
  def is_expired(self, now=None):



Analyst:
### Proposed Allocation

| Symbol | Allocation | Shares | Current Price | Value      |
|--------|------------|--------|---------------|------------|
| GOOGL  | 33.33%     | 37.48  | $320.18       | $11,998.81 |
| NVDA   | 33.33%     | 67.79  | $177.00       | $11,998.79 |
| COIN   | 33.34%     | 43.99  | $272.82       | $12,002.39 |

### 💎 Underrated Stocks to Consider

**MARA (Marathon Digital Holdings, Inc.)**

*   **Why?**: Marathon Digital is a significant player in the Bitcoin mining space. Its growth story is directly tied to the price of Bitcoin and the expanding cryptocurrency ecosystem. The company has been actively expanding its mining operations, which could lead to significant upside if Bitcoin experiences another bull run. The low trailing P/E suggests it might be undervalued based on past earnings.
*   **Why it was excluded**: MARA exhibits very high volatility (83.61%), indicating a higher risk profile compared to the core portfolio assets. It's also currently 

In [44]:
# --- Test the tool ---
print("Testing Portfolio Tool with $10k split between Apple, Nvidia, and Bitcoin...")
# Note: We pass a list of strings.
portfolio_result = allocate_portfolio(["AAPL", "NVDA", "BTC-USD"], 10000)
portfolio_result = json.dumps(portfolio_result, indent=4)
print(portfolio_result)

Testing Portfolio Tool with $10k split between Apple, Nvidia, and Bitcoin...
{
    "status": "success",
    "strategy": "Equal Weight",
    "total_investment": 10000,
    "allocation": [
        {
            "symbol": "AAPL",
            "price": 277.55,
            "shares": 12.0098,
            "value": 3333.32,
            "weight_allocated": "33.3%"
        },
        {
            "symbol": "NVDA",
            "price": 180.26,
            "shares": 18.4918,
            "value": 3333.33,
            "weight_allocated": "33.3%"
        },
        {
            "symbol": "BTC-USD",
            "price": 91048.76,
            "shares": 0.0366,
            "value": 3332.38,
            "weight_allocated": "33.3%"
        }
    ],
    "total_spent": 9999.03,
    "remaining_cash": 0.97
}


## Step 5: Final Agent Assembly
Updating the agent with the `allocate_portfolio` tool and refined instructions. The agent can now Research, Analyze Risk, AND Construct Portfolios in a single conversation.


In [52]:
# --- FINAL AGENT SETUP ---

final_tools = [
    get_asset_price_history,
    get_asset_fundamentals,
    calculate_technical_indicators,
    allocate_portfolio
]

final_instruction = """
You are a Proactive Wealth Manager.

YOUR GOAL:
Construct optimal portfolios for users, even if they don't know which stocks to pick.

RULES:
1. **Missing Tickers?**: If the user says "Invest $10k" but lists no assets:
   - Propose a mix yourself! (e.g., "I suggest a mix of Growth (NVDA), Stability (MSFT), and a small Crypto hedge (BTC)").
   - Choose 3-5 liquid, well-known assets appropriate for their implied risk.
   - Explain *why* you picked these specific assets (Sector diversification, growth potential).
   
2. **Process**:
   - **Step A**: Pick the assets (if not provided).
   - **Step B**: Analyze their Risk (Call `calculate_technical_indicators` for EACH).
   - **Step C**: Decide precise weights based on risk (Low Volatility = Higher Weight).
   - **Step D**: Call `allocate_portfolio` with your chosen assets and weights.

3. **Output**:
   - **Proposed Strategy**: "I selected [List] because..."
   - **Risk Analysis**: Table of volatility.
   - **Key Terms**: Define "Diversification" or "Beta" if relevant.
   - **Allocation**: Final table with precise weights.

DISCLAIMER: "This is a simulation for educational purposes."
"""

final_model = genai.GenerativeModel(
    model_name='gemini-2.5-flash',
    tools=final_tools,
    system_instruction=final_instruction
)

analyst_desk = final_model.start_chat(enable_automatic_function_calling=True)

print("Fully Autonomous Analyst Desk Ready!")
print("Type 'exit' to stop.\n")

# --- INTERACTIVE CHAT LOOP ---
while True:
    user_input = input("👤 You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
        
    print("Agent is thinking...")
    try:
        response = analyst_desk.send_message(user_input)
        print(f"\nAnalyst:\n{response.text}\n")
        print("-" * 80)
    except Exception as e:
        print(f"Error: {e}")


Fully Autonomous Analyst Desk Ready!
Type 'exit' to stop.



Agent is thinking...
Error: Invalid input: 'content' argument must not be empty. Please provide a non-empty value.
Agent is thinking...
Error: Invalid input: 'content' argument must not be empty. Please provide a non-empty value.
Agent is thinking...

Analyst:
Here is your proposed portfolio allocation:

**Allocation Plan:**

| Symbol | Shares | Value ($) | Weight Allocated |
| :----- | :----- | :-------- | :--------------- |
| NVDA   | 19.77  | 3500.00   | 35.0%            |
| TSLA   | 5.81   | 2500.02   | 25.0%            |
| GOOGL  | 12.49  | 4000.01   | 40.0%            |

This allocation aims to balance your desire for high returns and future value with a risk-adjusted approach, giving a slightly higher weight to the less volatile asset (GOOGL).

**Disclaimer:** This is a simulation for educational purposes. Investment in the stock market involves risks, and the value of investments can go down as well as up.

-----------------------------------------------------------------------